In [21]:
import csv
import pandas as pd

In [22]:
def apple_or_pen(transaction): # яблоко или ручка ?
    if "ap" in transaction:
        return True
    else:
        return False

In [23]:
def end_of_month(date1, date2): # последний день месяца ?
    if ((int(date1)-int(date2)) != 1):
        return True
    else:
        return False

In [24]:
def main(store):
    store_sell = pd.read_csv('MS-'+store+'-sell.csv')
    store_supply = pd.read_csv('MS-'+store+'-supply.csv')
    store_inventory = pd.read_csv('MS-'+store+'-inventory.csv')
    
    sold_apple = 0 # продано яблок
    sold_pen = 0 # продано ручек
    current_apple = 0 # текущее количество яблок 
    current_pen = 0 # текущее количество ручек
    supply_apple = 0 # поставлено яблок
    supply_pen = 0 # поставлено ручек
    stolen_apple = 0 # украдено яблок
    stolen_pen = 0 # украдено ручек
    
    first_supply = False # была ли поставка 1-го числа
    second_supply = False # была ли поставка 15-го числа
    
    daily_index = 0 # индекс в таблице ежедневных продаж
    stolen_index = 0 # индекс в таблице месячных краж
    supply_index = 0 # индекс в таблице поставок
    inventory_index = 0 # индекс в таблице месячного инвентаря
    i = 0 # индекс в таблице продаж
    
    curr_date = store_sell['date'][0] #текущую дату на 2006-01-01
#    last_date = store_sell['date'][store_sell['date'].shape[0]-1]
    
    DF_daily = pd.DataFrame(columns = ['date', 'apple', 'pen']) # дневные продажи
    DF_stolen = pd.DataFrame(columns = ['date', 'apple', 'pen']) # месячные кражи
    
    for i in range(store_sell['date'].shape[0]): # пока комп не накроется
        if (store_sell['date'][i] != curr_date): # транзакции поплыли на другой день
                    
            DF_total.loc[curr_date[:4], 'apple_sold'] += sold_apple # заносим проданные яблоки и ручки в 
            DF_total.loc[curr_date[:4], 'pen_sold'] += sold_pen # агрегированную таблицу
            
            current_apple = current_apple + supply_apple - sold_apple # осталось на конец текущего дня
            current_pen = current_pen + supply_pen - sold_pen
            
            if (end_of_month(store_sell['date'][i][8:10], curr_date[8:10])): # пора проверять "заимствованное" ?
                stolen_apple = current_apple - store_inventory['apple'][inventory_index] # украденные яблоки
                stolen_pen = current_pen - store_inventory['pen'][inventory_index] # украденные ручки
                DF_stolen.loc[stolen_index] = [curr_date, stolen_apple, stolen_pen] # заносим в таблицу все украденное
                DF_total.loc[curr_date[:4], 'apple_stolen'] += stolen_apple # обновляем записи в АТ
                DF_total.loc[curr_date[:4], 'pen_stolen'] += stolen_pen
                stolen_index += 1
                current_apple = store_inventory['apple'][inventory_index] # возвращаемся к суровой реальности
                current_pen = store_inventory['pen'][inventory_index]
                inventory_index += 1
            
            DF_daily.loc[daily_index] = [curr_date, current_apple, current_pen]
            
            daily_index += 1
            supply_apple = 0
            supply_pen = 0
            first_supply = False
            second_supply = False
            sold_apple = 0
            sold_pen = 0
            curr_date = store_sell['date'][i]

        if (store_sell['date'][i] == curr_date): # следущая транзацкия все еще на текущий день
            if (store_sell['date'][i][8:10] == "01") and (first_supply == False): # если сегодня 1 и пока не было поставки
                supply_apple += store_supply['apple'][supply_index]
                supply_pen += store_supply['pen'][supply_index]
                supply_index += 1
                first_supply = True
            if (store_sell['date'][i][8:10] == "15") and (second_supply == False): # если сегодня 15 и пока не было поставки      
                supply_apple += store_supply['apple'][supply_index]
                supply_pen += store_supply['pen'][supply_index]
                supply_index += 1
                second_supply = True
            if (apple_or_pen(store_sell['sku_num'][i])):
                sold_apple += 1
            else:
                sold_pen += 1
                
    
    # заносим данные за последний день продаж 
    DF_total.loc[curr_date[:4], 'apple_sold'] += sold_apple
    DF_total.loc[curr_date[:4], 'pen_sold'] += sold_pen         
    current_apple = current_apple + supply_apple - sold_apple
    current_pen = current_pen + supply_pen - sold_pen
    stolen_apple = current_apple - store_inventory['apple'][inventory_index]
    stolen_pen = current_pen - store_inventory['pen'][inventory_index]
    DF_stolen.loc[stolen_index] = [curr_date, stolen_apple, stolen_pen]
    DF_total.loc[curr_date[:4], 'apple_stolen'] += stolen_apple
    DF_total.loc[curr_date[:4], 'pen_stolen'] += stolen_pen
    current_apple = store_inventory['apple'][inventory_index]
    current_pen = store_inventory['pen'][inventory_index]      
    DF_daily.loc[daily_index] = [curr_date, current_apple, current_pen]
    
    DF_daily.to_csv('/home/maria/Документы/Практикум/6 семестр/artem-pyanykh-cmc-prac-task3-seed17/out/MS-'+store+'-daily.csv', index = False)
    DF_stolen.to_csv('/home/maria/Документы/Практикум/6 семестр/artem-pyanykh-cmc-prac-task3-seed17/out/MS-'+store+'-stolen.csv', index = False)

In [25]:
stores = ['b1','b2','m1', 
          'm2','s1','s2',
          's3','s4','s5'];

years = ['2006', '2007', '2008',
         '2009', '2010', '2011',
         '2012', '2013', '2014', '2015'];

sales = {'state': ['MS'],
         'apple_sold': [0*10],
         'apple_stolen': [0*10],
         'pen_sold': [0*10],
         'pen_stolen': [0*10]};

DF_total = pd.DataFrame(index = years, data = sales)

In [26]:
main('s1')

#for store in stores: #обход всех магазинов 
#   main(store)

In [27]:
DF_total.to_csv('/home/maria/Документы/Практикум/6 семестр/artem-pyanykh-cmc-prac-task3-seed17/out/MS-total.csv', index = False)
DF_total

,state,apple_sold,apple_stolen,pen_sold,pen_stolen
2006,MS,33930,31,3683,26
2007,MS,33608,25,3729,13
2008,MS,34071,19,3658,23
2009,MS,33671,33,3657,33
2010,MS,33806,31,3760,22
2011,MS,33918,30,3676,27
2012,MS,34046,19,3670,29
2013,MS,33729,19,3521,25
2014,MS,33871,24,3549,27
2015,MS,34099,17,3629,24
